In [ ]:
import pandas as pd
import pyam
import nomenclature
import ixmp4

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Current CDR",
    scenario="Observed Data",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
data = pd.read_excel(
    "source/7_Deployment/Indicator 15_Novel CDR volumes by method, year, and country.xlsx.xlsx",
    sheet_name="Dashboard",
    header=28,
    nrows=8,
    usecols="B:J",
).set_index("Year")

In [ ]:
df_novel = pyam.IamDataFrame(
    data,
    **iamc_args,
    value=data.columns,
    unit="t CO2",
    region="World",
)

In [ ]:
df_novel.rename(
    variable={
        "BECCS": "Bioenergy with CCS",
        "Bio-oil Storage": "Bio-Oil Storage",
        "DACCS": "Direct Air Capture with CCS",
        "ERW": "Enhanced Weathering",
        "Macroalgae": "Biomass Sinking",
        "Mineralisation": "Mineral Products",
    },
    inplace=True,
)
df_novel.rename(
    variable=dict([(i, f"Carbon Removal|Novel|{i}") for i in df_novel.variable]),
    inplace=True,
)
df_novel.aggregate(f"Carbon Removal|Novel", append=True)

In [ ]:
df_novel.rename(unit={"t CO2": "t CO2/yr"}, inplace=True)

In [ ]:
df_novel.convert_unit("t CO2/yr", "Mt CO2/yr", inplace=True)

In [ ]:
data = (
    pd.read_excel("source/7_Deployment/Data_Chapter7_Figure2.xlsx")
    .rename(columns={"Unnamed: 0": "year"})
    .set_index("year")
)
data

In [ ]:
df_conventional = pyam.IamDataFrame(
    data,
    **iamc_args,
    value=data.columns,
    unit="Mt CO2/yr",
    region="World",
)

In [ ]:
df_conventional.rename(variable={"Mean of models": "Mean of Models"}, inplace=True)

In [ ]:
df_conventional.variable

In [ ]:
df_conventional.rename(
    variable= dict(
        [(i, f"Carbon Flux|Conventional CDR [{i}]") for i in ["BLUE", "H&C2023", "OSCAR", "Mean of Models"]]
    ),
    inplace=True,
)

In [ ]:
nghgi_mapping = {
    "NGHGIs (lower-bound)": "Minimum",
    "NGHGIs (mean)": "Median",
    "NGHGIs (upper-bound)": "Maximum",
}

df_conventional.rename(
    variable= dict(
        [
            (key, f"Carbon Flux|Conventional CDR [Model-Adjusted Inventories][{value}]")
             for key, value in nghgi_mapping.items()
        ]
    ),
    inplace=True,
)

In [ ]:
df_conventional.convert_unit("Mt CO2/yr", "Gt CO2/yr", inplace=True)

In [ ]:
df = pyam.concat([df_novel, df_conventional])

In [ ]:
dsd.validate(df)

In [ ]:
df.set_meta("Chapter 7", "Chapter")

In [ ]:
# df.to_ixmp4("socdr-dev")

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(df.data)